In [1]:
import requests
from pyambit.datamodel import Substances, Study 
import nexusformat.nexus.tree as nx
import os.path
import tempfile
# to_nexus is not added without this import
from pyambit import nexus_writer
import json
from IPython.display import display, HTML

In [2]:
def query(url = "https://apps.ideaconsult.net/gracious/substance/" ,params = {"max" : 1}):
    substances = None
    headers = {'Accept': 'application/json'}
    result = requests.get(url,params=params,headers=headers)
    if result.status_code==200:
        response =  result.json()
        substances = Substances.model_construct(**response)
        for substance in substances.substance:
            url_study = "{}/study?max=10000".format(substance.URI)
            study = requests.get(url_study,headers=headers)
            if study.status_code==200:
                response_study = study.json()
                substance.study = Study.model_construct(**response_study).study
            #break

    return substances

def write_studies_nexus(substances, single_file=True):
    if single_file:
        nxroot = nx.NXroot()
        substances.to_nexus(nxroot)
        file = os.path.join(tempfile.gettempdir(), "remote.nxs")
        print(file)
        nxroot.save(file, mode="w")
    else:        
        for substance in substances.substance:
            for study in substance.study:
                file = os.path.join(tempfile.gettempdir(), "study_{}.nxs".format(study.uuid))
                print(file)
                nxroot = nx.NXroot()
                try:
                    study.to_nexus(nxroot)
                    nxroot.save(file, mode="w")
                except Exception as err:
                    #print("error",file,str(err))
                    print(file)

In [3]:
import traceback

In [5]:
url = "https://apps.ideaconsult.net/gracious/substance/"
#url = "http://localhost:9090/ambit2/substance/"
#url = "http://localhost:9090/ambit2/substance/POLY-e02442cc-8f7c-3a71-82cf-7df5888a4bfa"
#url = "http://localhost:9090/ambit2/substance/POLY-25d13fa6-c18b-35c8-b0f6-7325f5f3e505"
try:
    substances = query(url=url,params = {"max" : 1})   
    _json = substances.model_dump(exclude_none=True)
    new_substances = Substances.model_construct(**_json)
    #test roundtrip
    assert substances == new_substances

    file = os.path.join(tempfile.gettempdir(), "remote.json")
    print(file)
    with open(file, 'w', encoding='utf-8') as file:
        file.write(substances.model_dump_json(exclude_none=True))
    
    for s in substances.substance:
        for pa in s.study:
            effectarrays_only, df = pa.convert_effectrecords2array()
            display(df.dropna(axis=1,how="all"))
            print(effectarrays_only)
            #break
    #write_studies_nexus(substances, single_file=False)
except Exception as x:
    traceback.print_exc()


C:\Users\jelia\AppData\Local\Temp\remote.json
warning>  Float value REPLICATE:nan


Traceback (most recent call last):
  File "C:\Users\jelia\AppData\Local\Temp\ipykernel_13992\3684429070.py", line 19, in <module>
    effectarrays_only, df = pa.convert_effectrecords2array()
                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\nina\src\pyambit\src\pyambit\datamodel.py", line 1124, in convert_effectrecords2array
    earray = EffectArray(
             ^^^^^^^^^^^^
  File "d:\nina\src\pyambit\.venv\Lib\site-packages\pydantic\main.py", line 193, in __init__
    self.__pydantic_validator__.validate_python(data, self_instance=self)
pydantic_core._pydantic_core.ValidationError: 1 validation error for EffectArray
conditions
  Value error, cannot convert float NaN to integer [type=value_error, input_value={'END_DATE': '2020-12-15'...ART_DATE': '2020-12-15'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error
